In [13]:
!pip install gymnasium==0.29.1

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.28.1
    Uninstalling gymnasium-0.28.1:
      Successfully uninstalled gymnasium-0.28.1


In [1]:
import gymnasium as gym
gym.__version__

'0.29.1'

In [1]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
import time
from stable_baselines3.common.env_checker import check_env

env = gym.make("TwsbrEnv-v0",
                render_mode="human",
                action_type="continuous",
                )
check_env(env)
env.close()

In [1]:
#RUN JUST ONCE
from gymnasium.envs.registration import register
register(
    id="TwsbrEnv-v0",
    entry_point="twsbr_env.envs:TwsbrEnv",
)

In [2]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
import time

In [3]:
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous")

In [5]:
obs, info = env.reset()
obs, info

(array([0.        , 0.04285714, 0.        , 0.        , 0.        ],
       dtype=float32),
 {'step_count': 0})

In [5]:
env.action_space

Box(-1.0, 1.0, (2,), float32)

In [5]:
env.observation_space

Box(-1.0, 1.0, (5,), float32)

In [6]:
obs, info = env.reset()
obs, info

(array([ 0.     , -0.09375,  0.     ,  0.     ,  0.     ], dtype=float32),
 {'step_count': 0})

In [4]:
action = env.action_space.sample()
action

array([-0.6364568 ,  0.35971627], dtype=float32)

In [7]:
import numpy as np

In [8]:
action = np.array([0.01, -0.01]).astype(np.float32)

In [16]:
obs, reward, terminated, truncated, info = env.step(action=action)
obs, reward, terminated, truncated, info

(array([-3.7118336e-03, -9.3750000e-02,  1.2925364e-07,  1.3489862e-02,
        -1.3489604e-02], dtype=float32),
 np.float64(0.42753816644350695),
 False,
 False,
 {'step_count': 7})

##This part for the testing

In [1]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment

# Inisialisasi lingkungan
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous", robot_angle_limit = 45)

# Mulai episode inferensi
obs, info = env.reset()
terminated = False
truncated = False
for i in range(100000):
    # Pilih tindakan berdasarkan model
    action = env.action_space.sample()
    # Lakukan aksi di lingkungan
    obs, reward, terminated, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    if terminated or truncated:
        obs, info = env.reset()
    env.render()

# Menutup lingkungan setelah selesai
env.close()

error: Not connected to physics server.

In [1]:
env.close()

NameError: name 'env' is not defined

In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy

# Create the environment with specific parameters
env = gym.make("TwsbrEnv-v0",
                render_mode=None,
                action_type="continuous",  # Action type
                )

# Check if the environment is working as expected
from stable_baselines3.common.env_checker import check_env
check_env(env)

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(
    activation_fn=th.nn.LeakyReLU,
    net_arch=dict(pi=[64, 256], qf=[256, 256])  # Define architecture
)

# Create the agent (SAC - Soft Actor-Critic)
model = SAC("MlpPolicy", 
            env,  # Pass the environment here
            policy_kwargs=policy_kwargs,  # Pass custom policy kwargs
            learning_rate=0.0005,  # Define learning rate
            verbose=1)  # Verbosity level

# Train the agent
model.learn(total_timesteps=25_000)

# Save the agent after training
model.save("sac_tws_v3")

# Evaluation parameters
n_eval_episodes = 10
render = False  # Set to True if you want to render the evaluation

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()  # Get the environment from the model
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.25     |
|    ep_rew_mean     | -11.4    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 5        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.25     |
|    ep_rew_mean     | -11.3    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 10       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.67     |
|    ep_rew_mean     | -12.1    |
|    success_rate    

In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy



# Create the environment with specific parameters
env = gym.make("TwsbrEnv-v0",
                render_mode=None,
                robot_version="RV-1",  # Your specific robot version
                target_mode="self-balancing",  # Target mode
                action_type="continuous",  # Action type
                action_mode="AC-PWM"  # Action mode
                )

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], vf=[256, 256]))

# Create the agent
model = A2C("MlpPolicy", env, policy_kwargs=policy_kwargs, learning_rate=0.0005, verbose=1)

# Train the agent
model.learn(total_timesteps=50_000)

# Save the agent
model.save("a2c_tws")

# Evaluation parameters
n_eval_episodes = 10
render = False

# Evaluate agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
        
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

    # Cleanup
del model

Using cuda device
Creating environment from the given name 'TwsbrEnv-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 39.8       |
|    ep_rew_mean        | -54.8      |
|    success_rate       | 0          |
| time/                 |            |
|    fps                | 40         |
|    iterations         | 100        |
|    time_elapsed       | 12         |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -2.77      |
|    explained_variance | 0.70030123 |
|    learning_rate      | 0.0005     |
|    n_updates          | 99         |
|    policy_loss        | 2.13       |
|    std                | 0.965      |
|    value_loss         | 1.19       |
--------------------------------------
---------------------------------------
| rollout/              |             |
|    ep_len_mean        | 36.

In [2]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import A2C


# Memuat model yang telah dilatih
model = A2C.load("a2c_tws")

# Inisialisasi lingkungan
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous", robot_angle_limit = 45)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
for i in range(100000):
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=True)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    if done:
        obs, info = env.reset()
    env.render()

# Menutup lingkungan setelah selesai
env.close()

In [8]:
env.close()

error: Not connected to physics server.

In [4]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy

# Create the environment with specific parameters
env = gym.make("TwsbrEnv-v0",
                render_mode=None,
                robot_version="RV-3",  # Your specific robot version
                target_mode="self-balancing",  # Target mode
                action_type="continuous",  # Action type
                action_mode="AC-PWM"  # Action mode
                )

# Check if the environment is working as expected
from stable_baselines3.common.env_checker import check_env
check_env(env)

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(
    activation_fn=th.nn.LeakyReLU,
    net_arch=dict(pi=[256, 256], qf=[256, 256])  # Define architecture
)

# Create the agent (SAC - Soft Actor-Critic)
model = SAC("MlpPolicy", 
            env,  # Pass the environment here
            policy_kwargs=policy_kwargs,  # Pass custom policy kwargs
            learning_rate=0.0005,  # Define learning rate
            verbose=1)  # Verbosity level

# Train the agent
model.learn(total_timesteps=100_000)

# Save the agent after training
model.save("sac_tws_v3")

# Evaluation parameters
n_eval_episodes = 10
render = False  # Set to True if you want to render the evaluation

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()  # Get the environment from the model
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -205     |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 90       |
|    time_elapsed    | 44       |
|    total_timesteps | 4000     |
| train/             |          |
|    actor_loss      | -5.49    |
|    critic_loss     | 0.26     |
|    ent_coef        | 0.165    |
|    ent_coef_loss   | -3       |
|    learning_rate   | 0.0005   |
|    n_updates       | 3899     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 967      |
|    ep_rew_mean     | -123     |
|    success_rate    | 0.875    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 78       |
|    time_elapsed    

#sac_tws_v1  : good with 64 x 64 arch

#sac_tws_v2 : good enough with 64x64 arch

#sac_tws_v3 : good enough with arch 64x64

In [6]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import SAC

# Create the environment with specific parameters
env = gym.make("TwsbrEnv-v0",
                render_mode="human",
                robot_version="RV-3",  # Your specific robot version
                target_mode="self-balancing",  # Target mode
                action_type="continuous",  # Action type
                action_mode="AC-PWM"  # Action mode
                )

# Memuat model yang telah dilatih
model = SAC.load("sac_tws_v3")
# Mulai episode inferensi
acc_reward = 0
obs, info = env.reset()
done = False
truncated = False
for i in range(100000):
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=False)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    #print(f"{reward}")
    acc_reward += reward
    if done:
        obs, info = env.reset()
    env.render()

print(f"REWARD : {acc_reward}")
# Menutup lingkungan setelah selesai
env.close()

REWARD : 4954236.127802092


In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy

# Create the environment with specific parameters
env = gym.make("TwsbrEnv-v0",
                render_mode=None,
                action_type="continuous",  # Action type
                )

# Check if the environment is working as expected
from stable_baselines3.common.env_checker import check_env
check_env(env)

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(
    activation_fn=th.nn.LeakyReLU,
    net_arch=dict(pi=[256, 256], qf=[256, 256])  # Define architecture
)

# Create the agent (SAC - Soft Actor-Critic)
model = SAC("MlpPolicy", 
            env,  # Pass the environment here
            policy_kwargs=policy_kwargs,  # Pass custom policy kwargs
            learning_rate=0.0005,  # Define learning rate
            verbose=1)  # Verbosity level

# Train the agent
model.learn(total_timesteps=100_000)

# Save the agent after training
model.save("sac_tws_v3")

# Evaluation parameters
n_eval_episodes = 10
render = False  # Set to True if you want to render the evaluation

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()  # Get the environment from the model
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.25     |
|    ep_rew_mean     | -50.2    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 5        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.12     |
|    ep_rew_mean     | -50.3    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 9        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.08     |
|    ep_rew_mean     | -50.3    |
|    success_rate    

In [4]:
env.close()

error: Not connected to physics server.

In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], qf=[256, 256]))

# Create the agent
model = PPO("MlpPolicy", "TwsbrEnv-v0", policy_kwargs=policy_kwargs, learning_rate=0.0005, verbose=1)

# Train the agent
model.learn(total_timesteps=50_000)

# Save the agent
model.save("ppo_tws")

# Evaluation parameters
n_eval_episodes = 10
render = False

# Evaluate agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
        
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model


Using cuda device
Creating environment from the given name 'TwsbrEnv-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.1     |
|    ep_rew_mean     | -9.96    |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 22       |
|    iterations      | 1        |
|    time_elapsed    | 91       |
|    total_timesteps | 2048     |
---------------------------------
--------------------------------------------
| rollout/                |                |
|    ep_len_mean          | 20.9           |
|    ep_rew_mean          | -11.8          |
|    success_rate         | 0              |
| time/                   |                |
|    fps                  | 21             |
|    iterations           | 2              |
|    time_elapsed         | 186            |
|    total_timesteps      | 4096           |
| train/                  

In [9]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import PPO


# Memuat model yang telah dilatih
model = PPO.load("ppo_tws")

# Inisialisasi lingkungan
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous", robot_angle_limit = 45)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
for i in range(1000):
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=False)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    if done or truncated:
        obs, info = env.reset()
    env.render()

# Menutup lingkungan setelah selesai
env.close()

In [10]:
env.close()

Exception ignored in: <function BulletClient.__del__ at 0x000002676B90ED40>
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\pybullet_utils\bullet_client.py", line 39, in __del__
    if self._client>=0 and self._pid == os.getpid():
  File "c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\pybullet_utils\bullet_client.py", line 48, in __getattr__
    attribute = getattr(pybullet, name)
AttributeError: module 'pybullet' has no attribute '_client'


error: Not connected to physics server.

In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv
from stable_baselines3 import DDPG
from stable_baselines3.common.evaluation import evaluate_policy

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], qf=[256, 256]))

# Create the agent
model = DDPG("MlpPolicy", "TwsbrEnv-v0", policy_kwargs=policy_kwargs, learning_rate=0.0005, verbose=1)

# Train the agent
model.learn(total_timesteps=50_000)

# Save the agent
model.save("ddpg_tws")

# Evaluation parameters
n_eval_episodes = 10
render = False

# Evaluate agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
        
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model


Using cuda device
Creating environment from the given name 'TwsbrEnv-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 14.8     |
|    ep_rew_mean     | -13.6    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 14       |
|    time_elapsed    | 4        |
|    total_timesteps | 59       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17       |
|    ep_rew_mean     | -13      |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 18       |
|    time_elapsed    | 7        |
|    total_timesteps | 136      |
| train/             |          |
|    actor_loss      | 0.736    |
|    critic_loss     | 0.103    |
|    learning_rate   | 0.0005   |


KeyboardInterrupt: 

In [13]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import DDPG


# Memuat model yang telah dilatih
model = DDPG.load("ddpg_tws")

# Inisialisasi lingkungan
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous", robot_angle_limit = 45)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
for i in range(100000):
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=True)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    if done:
        obs, info = env.reset()
    env.render()

# Menutup lingkungan setelah selesai
env.close()

In [ ]:
import gymnasium as gym
import torch as th
from twsbr_env.envs import TwsbrEnv
from stable_baselines3 import TD3
from stable_baselines3.common.evaluation import evaluate_policy

# Custom actor (pi) and value function (vf) networks
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], qf=[256, 256]))

# Create the agent
model = TD3("MlpPolicy", "TwsbrEnv-v0", policy_kwargs=policy_kwargs, learning_rate=0.0005, verbose=1)

# Train the agent
model.learn(total_timesteps=50_000)

# Save the agent
model.save("td3_tws")

# Evaluation parameters
n_eval_episodes = 10
render = False

# Evaluate agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

# Custom evaluation metrics
env = model.get_env()
total_rewards = []
episode_lengths = []

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        episode_length += 1
        
    total_rewards.append(episode_reward)
    episode_lengths.append(episode_length)

# Print evaluation metrics
print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Average Episode Length: {sum(episode_lengths) / n_eval_episodes:.2f}")
print(f"Success Rate: {sum([1 for r in total_rewards if r >= 200]) / n_eval_episodes * 100:.2f}%")  # Assuming success threshold of 200

# Cleanup
del model


Using cuda device
Creating environment from the given name 'TwsbrEnv-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.2     |
|    ep_rew_mean     | -8.99    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 16       |
|    time_elapsed    | 5        |
|    total_timesteps | 97       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.2     |
|    ep_rew_mean     | -7.27    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 17       |
|    time_elapsed    | 11       |
|    total_timesteps | 194      |
| train/             |          |
|    actor_loss      | 0.265    |
|    critic_loss     | 0.719    |
|    learning_rate   | 0.0005   |


In [14]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
from stable_baselines3 import TD3


# Memuat model yang telah dilatih
model = TD3.load("td3_tws")

# Inisialisasi lingkungan
env = gym.make('TwsbrEnv-v0', render_mode="human", action_type="continuous", robot_angle_limit = 45)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
for i in range(100000):
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=True)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    if done:
        obs, info = env.reset()
    env.render()

# Menutup lingkungan setelah selesai
env.close()

error: Not connected to physics server.

#PID MODE

In [13]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
import numpy as np
import time
import matplotlib.pyplot as plt

env = gym.make("TwsbrEnv-v0", render_mode="human")
obs, info = env.reset()

# PID controller parameters for balance, steering, and distance
Kp_balance = 25.0
Kd_balance = 0.1
Ki_balance = 0.001

Kp_steer = 0.1
Kd_steer = 0.01
Ki_steer = 0.001

Kp_distance = 0.1
Kd_distance = 0.01
Ki_distance = 0.001

# Variables for PID calculations
integral_balance = 0.0
prev_error_balance = 0.0

integral_steer = 0.0
prev_error_steer = 0.0

integral_distance = 0.0
prev_error_distance = 0.0

# Define target positions
target_positions = [(1, 0, 0), (2, 0, 0), (2, 1, 0), (2, 2, 0)]
current_target_index = 0

# Simulation parameters
dt = 1. / 240.
t = np.arange(0, 30, dt)
states = []

# Functions for PID calculations
def pid_distance(current_position, target_position):
    global integral_distance, prev_error_distance
    error = np.linalg.norm(np.array(target_position[:2]) - np.array(current_position[:2]))  # Distance error in x-y plane
    integral_distance += error * dt
    derivative = (error - prev_error_distance) / dt
    prev_error_distance = error
    return Kp_distance * error + Ki_distance * integral_distance + Kd_distance * derivative

def pid_balance(current_angle, target_angle=0.0):
    global integral_balance, prev_error_balance
    error = target_angle - current_angle
    integral_balance += error * dt
    derivative = (error - prev_error_balance) / dt
    prev_error_balance = error
    return Kp_balance * error + Ki_balance * integral_balance + Kd_balance * derivative

def pid_steer(current_angle, target_angle):
    global integral_steer, prev_error_steer
    error = target_angle - current_angle
    integral_steer += error * dt
    derivative = (error - prev_error_steer) / dt
    prev_error_steer = error
    return Kp_steer * error + Ki_steer * integral_steer + Kd_steer * derivative

# Function to check if the robot has reached the target
def has_reached_target(current_position, target_position, threshold=0.1):
    distance = np.linalg.norm(np.array(target_position[:2]) - np.array(current_position[:2]))
    return distance < threshold

for time_step in t:
    # Get the current target position
    target_position = target_positions[current_target_index]
    
    # Calculate distance and steering actions
    distance_action = pid_distance(obs[:2], target_position)
    steering_action = pid_steer(obs[5], np.arctan2(target_position[1] - obs[1], target_position[0] - obs[0]))
    
    # Calculate balance action to keep the robot upright
    balance_action = pid_balance(obs[1], 0.0)

    # Combine actions
    left_wheel_action = balance_action + distance_action - steering_action
    right_wheel_action = balance_action + distance_action + steering_action
    action = np.array([left_wheel_action, right_wheel_action])
    
    # Perform action in environment
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    
    # Check if the robot has reached the current target position
    if has_reached_target(obs[:2], target_position):
        current_target_index = (current_target_index + 1) % len(target_positions)  # Move to next target

    # Check for termination or truncation
    if terminated or truncated:
       obs, info = env.reset()

env.close()


error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.

In [14]:
env.close()

error: Not connected to physics server.

In [2]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
import time

# Fungsi PID
class PIDController:
    def __init__(self, kp, ki, kd, setpoint=0.0):
        self.kp = kp  # Gain Proporsional
        self.ki = ki  # Gain Integral
        self.kd = kd  # Gain Derivatif
        self.setpoint = setpoint  # Titik referensi yang diinginkan (0 untuk keseimbangan)
        self.integral = 0.0  # Menyimpan kumulatif error untuk komponen integral
        self.previous_error = 0.0  # Menyimpan error sebelumnya untuk komponen derivatif
        self.last_time = None  # Waktu saat iterasi terakhir untuk menghitung dt

    def compute(self, current_value):
        # Hitung error antara setpoint dan nilai saat ini
        error = self.setpoint - current_value

        # Waktu saat ini untuk menghitung dt
        current_time = time.time()
        dt = current_time - self.last_time if self.last_time else 0.01
        self.last_time = current_time

        # Komponen Integral
        self.integral += error * dt

        # Komponen Derivatif
        derivative = (error - self.previous_error) / dt if dt > 0 else 0.0
        self.previous_error = error

        # Komponen Proporsional, Integral, dan Derivatif digabung
        output = self.kp * error + self.ki * self.integral + self.kd * derivative

        return output


# Inisialisasi lingkungan
env = gym.make("TwsbrEnv-v0", render_mode="human", robot_version="RV-1", robot_angle_limit=30)

# Inisialisasi pengendali PID
# Parameter ini perlu disesuaikan berdasarkan respon robot Anda
pid_controller = PIDController(kp=10, ki=0.0001, kd=0.1)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
for episode in range(100000):
    # Asumsi bahwa obs berisi informasi sudut kemiringan robot pada indeks tertentu, misalnya obs[0]
    angle = obs[1]  # Sudut kemiringan robot yang ingin distabilkan (contoh saja)

    # Hitung aksi kontrol berdasarkan nilai sudut menggunakan PID
    control_action = pid_controller.compute(angle)

    # Sesuaikan aksi untuk lingkungan (misal, aksi terukur dalam batas lingkungan)
    action = [-control_action, -control_action]  # Sesuaikan bentuk sesuai action_space TwsbrEnv

    # Lakukan aksi di lingkungan
    obs, reward, terminated, truncated, info = env.step(action)
    
    # Render visualisasi hasilnya
    env.render()

    # Reset environment jika episode selesai
    if terminated or truncated:
        obs, info = env.reset()

# Menutup lingkungan setelah selesai
env.close()


error: Not connected to physics server.

In [1]:
import gymnasium as gym
from twsbr_env.envs import TwsbrEnv  # Import environment
import time
import matplotlib.pyplot as plt

# Fungsi PID
class PIDController:
    def __init__(self, kp, ki, kd, setpoint=0.0):
        self.kp = kp  # Gain Proporsional
        self.ki = ki  # Gain Integral
        self.kd = kd  # Gain Derivatif
        self.setpoint = setpoint  # Titik referensi yang diinginkan (0 untuk keseimbangan)
        self.integral = 0.0  # Menyimpan kumulatif error untuk komponen integral
        self.previous_error = 0.0  # Menyimpan error sebelumnya untuk komponen derivatif
        self.last_time = None  # Waktu saat iterasi terakhir untuk menghitung dt

    def compute(self, current_value):
        # Hitung error antara setpoint dan nilai saat ini
        error = self.setpoint - current_value

        # Waktu saat ini untuk menghitung dt
        current_time = time.time()
        dt = current_time - self.last_time if self.last_time else 0.01
        self.last_time = current_time

        # Komponen Integral
        self.integral += error * dt

        # Komponen Derivatif
        derivative = (error - self.previous_error) / dt if dt > 0 else 0.0
        self.previous_error = error

        # Komponen Proporsional, Integral, dan Derivatif digabung
        output = self.kp * error + self.ki * self.integral + self.kd * derivative

        return output


# Inisialisasi lingkungan
env = gym.make("TwsbrEnv-v0", render_mode="human")

# Inisialisasi pengendali PID
pid_controller = PIDController(kp=0.75, ki=0.01, kd=0.1)

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False

# Data untuk grafik
pitch_data = []
time_data = []
start_time = time.time()

for episode in range(1000):
    # Asumsi bahwa obs berisi informasi sudut kemiringan robot pada indeks tertentu, misalnya obs[1]
    pitch = obs[1]  # Sudut kemiringan robot yang ingin distabilkan (contoh saja)
    
    # Hitung aksi kontrol berdasarkan nilai sudut menggunakan PID
    control_action = pid_controller.compute(pitch)
    
    # Sesuaikan aksi untuk lingkungan (misal, aksi terukur dalam batas lingkungan)
    action = [-control_action, -control_action]  # Sesuaikan bentuk sesuai action_space TwsbrEnv

    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    
    # Simpan pitch angle dan waktu untuk plot
    pitch_data.append(pitch)
    time_data.append(time.time() - start_time)
    
    # Render visualisasi hasilnya
    env.render()

    # Reset environment jika episode selesai
    if done:
        obs, info = env.reset()

# Menutup lingkungan setelah selesai
env.close()

# Plot grafik pitch terhadap waktu
plt.plot(time_data, pitch_data, label="Pitch Angle")
plt.axhline(0, color="r", linestyle="--", label="Target Angle (Upright)")
plt.xlabel("Time (s)")
plt.ylabel("Pitch Angle (degrees)")
plt.title("Pitch Angle Stabilization using PID")
plt.legend()
plt.show()


error: Not connected to physics server.